In [ ]:
# Gensim
import gensim
from gensim.models import Word2Vec


# to make nbs importable
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
#from nbs_import import NotebookLoader

import pandas as pd
import numpy as np

# custom
from analize_text import get_sentenceID
from paths import *


# nltk
import nltk
from nltk import word_tokenize
from nltk.stem.snowball import EnglishStemmer
from nltk import pos_tag, pos_tag_sents

from multiprocessing import cpu_count

# scikit learn
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

# keras
from keras.utils import np_utils

from feature_transformer import *

### Loading the dataset (embeddings_POS)

In [ ]:
data_path = os.path.join(ROOT_DIR, 'XY', 'stemPOS')
X_test2 = np.load(os.path.join(data_path, 'X_test2.npy'))
X_test1 = np.load(os.path.join(data_path, 'X_test1.npy'))
X_train = np.load(os.path.join(data_path, 'X_train.npy'))

Y_test2 = np.load(os.path.join(data_path, 'Y_test2.npy'))
Y_test1 = np.load(os.path.join(data_path, 'Y_test1.npy'))
Y_train = np.load(os.path.join(data_path, 'Y_train.npy'))
print(X_test1.shape, Y_test1.shape)

In [ ]:
w2vmodel = Word2Vec.load('../word_vectors')
word_vectors = w2vmodel.wv

### Encode labels
- Convert labels from B-I-O to $0, 1, 2$ for SVM
- Convert labels from B-I-O to $[1 0 0, 0 1 0, 0 0 1]$ for ANN

In [ ]:
# encode class values as integers = B-I-O -> 0-1-2
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y_train)
Y_train = encoded_Y
# convert integers to one-hot encoding
Y_train_one_hot = np_utils.to_categorical(encoded_Y) # SVM does not need one-hot encoding

### Creating feature transformers

In [ ]:
class Embedder(FeatureTransformer):
  # returns embedding for each word
  def transform(self, X, y=None, **fit_params):
    return np.array([word_vectors[x] for x in X])

In [ ]:
emb = Embedder()
e = emb.transform(['conduction_NN', words_train[7]])
print(e.shape)

In [ ]:
class Capitalizer(FeatureTransformer):
    # return 1 if the word is capitalized
    def transform(self, X, y=None, **fit_params):
        return np.array([1 if x[0].isupper() else 0 for x in X]).reshape(-1,1)

In [ ]:
cap = Capitalizer()
c = cap.transform(['conduction_NN', words_train[7]])
print(c.shape)

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC, LinearSVC

clf = LinearSVC()

pipe = FeatureUnion([
    ('emb', Embedder()),
    ('cap', Capitalizer()),
])

print(words_train.reshape(-1, 1).shape)
data = pipe.transform(words_train.reshape(-1, 1))
print(data.shape)
#clf.fit(data)

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
example_sent = "This is a sample sentence, showing off the stop words filtration."
 
stop_words = set(stopwords.words('english'))
 
word_tokens = word_tokenize(example_sent)
 
filtered_sentence = [w for w in word_tokens if not w in stop_words]
 
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
 
print(word_tokens)
print(filtered_sentence)

In [ ]:
class PunctuationRemoval(FeatureTransformer):
    def transform():
        pass
        

# Data preprocessing
experiment with:
- original data
- MinMaxScaler
- Standardizer

In [ ]:
#min_max_scaler = MinMaxScaler()
#X_train = min_max_scaler.fit_transform(X_train)

#standard_scaler = StandardScaler()
#X_train = standard_scaler.fit_transform(X_train)

## split train / validation set

In [ ]:
# split train validation (SVM)
train_perc = 0.9
train_size = int(len(X_train) * train_perc)

X_tr, X_vl = X_train[:train_size,:], X_train[train_size:,:]
Y_tr, Y_vl = Y_train[:train_size], Y_train[train_size:]

print ("X train/validation shapes:", X_tr.shape, X_vl.shape)
print ("Y train/validation shapes (svm):", Y_tr.shape, Y_vl.shape)

# split train validatioin (NN)
Y_tr_nn, Y_vl_nn = Y_train_one_hot[:train_size], Y_train_one_hot[train_size:]
print ("Y train/validation shapes: (ann): ", Y_tr_nn.shape, Y_vl_nn.shape)

# Using SVM

In [ ]:
from sklearn import svm

# one-vs-all classifier
model = svm.SVC(kernel='rbf', 
                C=1.0,
                class_weight=None,
                gamma='auto',
                #penalty='l2',
                #loss='squared_hinge',
                tol=0.001, random_state=None)

In [ ]:
model.fit(X_tr, Y_tr)

#### Compute metrics
- accuracy
- f1-score (micro, macro, weighted)
- precision (micro, macro, weighted)
- recall (micro, macro, weighted)

In [ ]:
predictions = model.predict(X_vl)

# micro:
# computes metrics globally - considering total number of TP, FP, FN
f1_micro = f1_score(Y_vl, predictions, average='micro')
recall_micro = recall_score(Y_vl, predictions, average='micro')
precision_micro = precision_score(Y_vl, predictions, average='micro')

# macro:
# compute metrics for each label, then unweighted average them - does not take class imbalance into account
f1_macro = f1_score(Y_vl, predictions, average='macro')
recall_macro = recall_score(Y_vl, predictions, average='macro')
precision_macro = precision_score(Y_vl, predictions, average='macro')

# weighted:
# as macro, but taking into account true class frequencies for each label
f1_weighted = f1_score(Y_vl, predictions, average='weighted')
recall_weighted = recall_score(Y_vl, predictions, average='weighted')
precision_weighted = precision_score(Y_vl, predictions, average='weighted')

accuracy = accuracy_score(Y_vl, predictions)

In [ ]:
print ('SVM validation accuracy:', round(accuracy, 4))
print ('-'*20)
print ('SVM validation f1-score (micro):', round(f1_micro, 4))
print ('SVM validation precision (micro):', round(precision_micro, 4))
print ('SVM validation recall (micro):', round(recall_micro, 4))
print ('-'*20)
print ('SVM validation f1-score (macro):', round(f1_macro, 4))
print ('SVM validation precision (macro):', round(precision_macro, 4))
print ('SVM validation recall (macro):', round(recall_macro, 4))
print ('-'*20)
print ('SVM validation f1-score (weighted):', round(f1_weighted, 4))
print ('SVM validation precision (weighted):', round(precision_weighted, 4))
print ('SVM validation recall (weighted):', round(recall_weighted, 4))

# Output
IdSentence|startOffset-endOffset|text|null